# Testing Aer Simulator Methods

This notebook tests different Aer simulator methods with the Qonscious framework.

Available Aer simulation methods:
- **automatic**: Automatically select the best method
- **statevector**: State vector simulation
- **density_matrix**: Density matrix simulation
- **stabilizer**: Stabilizer simulation (Clifford circuits only)
- **matrix_product_state**: Matrix product state simulation
- **extended_stabilizer**: Extended stabilizer simulation
- **unitary**: Unitary matrix simulation
- **superop**: Superoperator simulation

## Setup and Imports

In [ ]:
import json
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler
from qonscious.adapters import AerSimulatorAdapter
from qonscious.checks import MeritComplianceCheck
from qonscious.foms import AggregateQPUFigureOfMerit
from qonscious.actions import QonsciousCallable
from qonscious import run_conditionally
import matplotlib.pyplot as plt

## Helper Functions

In [ ]:
def create_test_circuit(n_qubits=3):
    """Create a simple Bell-like test circuit."""
    qc = QuantumCircuit(n_qubits)
    qc.h(0)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    qc.measure_all()
    return qc

def on_pass(backend_adapter, figureOfMeritResults):
    """Executed if merit check passes - runs a circuit and returns result."""
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks PASSED ---")
    properties = firstFoMResult.get('properties', {})
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)
    num_qubits = properties.get('n_qubits', 'N/A')
    print(f"\nNumber of Qubits: {num_qubits}")
    print("-" * 25)
    
    # Execute a test circuit
    print("\n--- Running Test Circuit in on_pass ---")
    circuit = create_test_circuit(3)
    run_result = backend_adapter.run(circuit, shots=1000)
    print(f"Circuit executed successfully")
    print(f"Result counts: {run_result['counts']}")
    
    return run_result  # Return the experiment result

def on_fail(backend_adapter, figureOfMeritResults):
    """Executed if merit check fails."""
    firstFoMResult = figureOfMeritResults[0]
    print("\n--- Checks FAILED ---")
    properties = firstFoMResult.get('properties', {})
    print("\n--- Calculated Properties ---")
    pretty_properties = json.dumps(properties, indent=2)
    print(pretty_properties)
    return None

In [ ]:
def plot_results(qonscious_result, method_name=""):
    """Plot the measurement results from a qonscious result."""
    if qonscious_result is None or qonscious_result.get('experiment_result') is None:
        print("No experiment result to plot")
        return
    
    # Get the counts from the experiment result
    counts = qonscious_result['experiment_result']['counts']
    
    # For a 3-qubit circuit, we have 8 possible outcomes
    labels = ['000', '001', '010', '011', '100', '101', '110', '111']
    values = [counts.get(k, 0) for k in labels]
    
    # Calculate total shots
    N = sum(values)
    
    # Calculate fidelity (for Bell state, we expect mostly 000 and 111)
    fidelity = (counts.get('000', 0) + counts.get('111', 0)) / N if N else float('nan')
    title = f"3-Qubit Bell State - {method_name}\n(Fidelity = {fidelity:.3f})"
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(labels, values, color='steelblue', alpha=0.7)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel("Outcome", fontsize=12)
    ax.set_ylabel("Counts", fontsize=12)
    ax.set_ylim(bottom=0)
    ax.grid(True, axis='y', linestyle=':', linewidth=0.5, alpha=0.7)
    
    # Add value labels on top of each bar
    for bar, value in zip(bars, values):
        if value > 0:  # Only show non-zero values
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_height(),
                str(value),
                ha='center', va='bottom',
                fontsize=10
            )
    
    # Highlight the expected outcomes (000 and 111) in a different color
    bars[0].set_color('green')  # 000
    bars[0].set_alpha(0.8)
    bars[7].set_color('green')  # 111
    bars[7].set_alpha(0.8)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nTotal shots: {N}")
    print(f"Expected outcomes (000 + 111): {counts.get('000', 0) + counts.get('111', 0)} ({fidelity*100:.1f}%)")
    print(f"Unexpected outcomes: {N - counts.get('000', 0) - counts.get('111', 0)} ({(1-fidelity)*100:.1f}%)")


## Test Function for Each Method

In [ ]:
def test_aer_method(method_name, n_qubits=3, qubit_threshold=2):
    """
    Test a specific Aer simulator method.
    
    Args:
        method_name: Aer simulation method
        n_qubits: Number of qubits for the simulator
        qubit_threshold: Minimum qubits required to pass check
    """
    print(f"\n{'='*60}")
    print(f"Testing Aer Simulator Method: {method_name}")
    print(f"{'='*60}")
    
    try:
        # Create simulator with specific method
        simulator = AerSimulator(method=method_name)
        sampler = Sampler()
        
        # Create mock qubit properties (for local simulator)
        qubits_properties = [
            {"T1": [100e-6], "T2": [80e-6]} for _ in range(n_qubits)
        ]
        
        # Create adapter
        adapter = AerSimulatorAdapter(sampler, simulator, qubits_properties)
        
        print(f"\nSimulator Method: {method_name}")
        print(f"Available Qubits: {adapter.n_qubits}")
        
        # Create merit compliance check
        check_QPU = MeritComplianceCheck(
            figure_of_merit=AggregateQPUFigureOfMerit(),
            decision_function=lambda r: r is not None and r["properties"]["n_qubits"] > qubit_threshold
        )
        
        # Run conditional check
        qonscious_result = run_conditionally(
            backend_adapter=adapter,
            checks=[check_QPU],
            on_pass=QonsciousCallable(on_pass),
            on_fail=QonsciousCallable(on_fail)
        )
        
        
        return qonscious_result
        
    except Exception as e:
        print(f"\n❌ Error testing method '{method_name}': {str(e)}")
        return None

## Test All Methods at Once

In [ ]:
methods = [
    'automatic',
    'statevector',
    'density_matrix',
    'stabilizer',
    'matrix_product_state',
    'extended_stabilizer',
    'unitary',
    'superop'
]

print("\n" + "="*60)
print("TESTING ALL AER SIMULATOR METHODS")
print("="*60)

results = {}
for method in methods:
    result = test_aer_method(method, n_qubits=5, qubit_threshold=4)
    results[method] = result
    if result is not None and result.get('experiment_result') is not None:
        plot_results(result, method_name=method)
    else:
        print("No results to plot - make sure the test passed!")
    print("\n")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for method, result in results.items():
    status = "Success" if result is not None else "Failed"
    
    print(f"{method:25s} - {status}")